**This notebook we will try to create a model that accepts multiple inputs**

In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# other imports to handle files
import os
import pickle
import csv

# deep learning imports
from keras.models import Sequential, load_model, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, Convolution1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils, generic_utils, to_categorical
from keras.layers import LeakyReLU, Input, ReLU, concatenate
from keras import regularizers

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# change to directories with brainscans
os.chdir('C://Users//steve//Downloads/stored_processed_scans/processed_brains_aug/')

# load a set of brainscans to use
with open('dbatch0.pkl', 'rb') as f: # also 'total_slices_all.pkl' ## RENAMED 5 TO 7, TESTING IT
    total_slices, total_slices_info = pickle.load(f) # stored_batches/total_slices_batch5

In [13]:
def get_classification(total_slices_info):
    '''get information of patient'''
    classif = [s[2] for s in total_slices_info]
    classif = np.array(to_categorical(classif, 2))
    
    # now do the same for the sex, 'F' is 0, 'M' is 1
    sex = [s[1] for s in total_slices_info]
    for i in range(len(sex)):
        if sex[i] == 'F':
            sex[i] = 0
        if sex[i] == 'M':
            sex[i] = 1
    sex = np.array(to_categorical(sex, 2))
    
    # finally for age, one hot encode ages 0 to 100 (101 classes then)
    ages = [s[3] for s in total_slices_info]
    ages = np.array(to_categorical(ages, 101))
    
    return classif, sex, ages

y_values, sex, ages = get_classification(total_slices_info)

In [62]:
# creds: https://stackoverflow.com/questions/3674409/how-to-split-partition-a-dataset-into-training-and-test-datasets-for-e-g-cros

# split all 3 arrays into training and test sets
num_random = np.random.rand(len(y_values))
indices = np.random.permutation(y_values.shape[0])
training_idx, test_idx = indices[:90], indices[10:]

y_val_train, y_val_test = y_values[training_idx,:], y_values[test_idx,:]
sex_train, sex_test = sex[training_idx,:], sex[test_idx,:]
ages_train, ages_test = ages[training_idx,:], ages[test_idx,:]

In [45]:
def create_mlp(dim, regress=False):
    # define our MLP network
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu"))
    model.add(Dense(2, activation="relu"))
 
    # check to see if the regression node should be added
    if regress:
        model.add(Dense(1, activation="linear"))
 
    # return our model
    return model

In [46]:
# Create MLP models
mlp_sex = create_mlp(sex.shape[1], regress=False)
mlp_age = create_mlp(ages.shape[1], regress=False)

# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combined_input = concatenate([mlp_sex.output, mlp_age.output])

# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu")(combined_input)
x = Dense(2, activation="sigmoid")(x)

# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp_sex.input, mlp_age.input], outputs=x)

In [ ]:
# compile our model
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=1e-3, decay=1e-3 / 200), 
              metrics = ['categorical_accuracy']) # decay in Adam..

# train the model
print("training model...")
model.fit(
    [sex_train, ages_train], y_val_train,
    validation_split=0.1,
    epochs=200, batch_size=25)

In [64]:
# make predictions now
score, acc = model.evaluate([sex_test, ages_test], y_val_test)

# soooooooo this model is freakin powerful

90/90 [==============================] - 0s 113us/step


In [88]:
acc
# 81.1% can be our baseline, can see how it does when we average it across all the batches we want to train with 
# and test against an entire batch

0.8111111164093018

In [ ]:
model.summary()

***Above is fantastic, now ensemble with some cnn outputs***

In [91]:
# try this again here
model_cnn = initialise_cnn()

# combine again
combined_input2 = concatenate()

160

In [89]:
def initialise_cnn():

    # compile our model
    model = Sequential()

    model.add(Convolution3D(filters=16, kernel_size=3, padding='same', strides=1, input_shape=(160,160,160,1),
                          kernel_regularizer=regularizers.l2(0.005), bias_regularizer=regularizers.l2(0.005))) # padding on first one only?
    model.add(LeakyReLU(alpha=0.20)) # set to 0.01
    model.add(BatchNormalization(momentum=0.5))
    # could user he_norm kernel initializer?
    model.add(MaxPooling3D(pool_size=2, strides=2)) # pool_size=2

    model.add(Convolution3D(filters=32, kernel_size=3, padding='same', strides=1,
                          kernel_regularizer=regularizers.l2(0.005), bias_regularizer=regularizers.l2(0.005))) # padding on first one only?
    model.add(LeakyReLU(alpha=0.20)) # set to 0.01
    model.add(BatchNormalization(momentum=0.5))
    # could user he_norm kernel initializer?
    model.add(MaxPooling3D(pool_size=2, strides=2)) # pool_size=2

    model.add(Convolution3D(filters=64, kernel_size=3, padding='same', strides=1,
                          kernel_regularizer=regularizers.l2(0.005), bias_regularizer=regularizers.l2(0.005))) # padding on first one only?
    model.add(LeakyReLU(alpha=0.20)) # set to 0.01
    model.add(BatchNormalization(momentum=0.5))
    # could user he_norm kernel initializer?
    model.add(MaxPooling3D(pool_size=2, strides=2)) # pool_size=2

    model.add(Convolution3D(filters=128, kernel_size=3, padding='same', strides=2,
                          kernel_regularizer=regularizers.l2(0.005), bias_regularizer=regularizers.l2(0.005))) # padding on first one only?
    model.add(LeakyReLU(alpha=0.20)) # set to 0.01
    model.add(BatchNormalization(momentum=0.5))
    # could user he_norm kernel initializer?
    model.add(MaxPooling3D(pool_size=2, strides=2)) # pool_size=2

    model.add(Convolution3D(filters=256, kernel_size=3, padding='same', strides=2,
                          kernel_regularizer=regularizers.l2(0.005), bias_regularizer=regularizers.l2(0.005))) # padding on first one only?
    model.add(LeakyReLU(alpha=0.20)) # set to 0.01
    model.add(BatchNormalization(momentum=0.5))
    # could user he_norm kernel initializer?
    model.add(MaxPooling3D(pool_size=2, strides=2)) # pool_size=2

    model.add(Flatten())

    model.add(Dense(512, kernel_regularizer=regularizers.l2(0.005)))
    model.add(LeakyReLU(alpha=0.2)) # set to 0.01

    model.add(Dropout(0.20)) # add dropout to prevent overfitting

    model.add(Dense(64, kernel_regularizer=regularizers.l2(0.005)))
    model.add(LeakyReLU(alpha=0.2)) # set to 0.01

    model.add(Dropout(0.35)) # add dropout to prevent overfitting

    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',metrics = ['categorical_accuracy'])

    return model

In [90]:
model_cnn = initialise_cnn()
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_53 (Conv3D)           (None, 160, 160, 160, 16) 448       
_________________________________________________________________
leaky_re_lu_63 (LeakyReLU)   (None, 160, 160, 160, 16) 0         
_________________________________________________________________
batch_normalization_53 (Batc (None, 160, 160, 160, 16) 64        
_________________________________________________________________
max_pooling3d_53 (MaxPooling (None, 80, 80, 80, 16)    0         
_________________________________________________________________
conv3d_54 (Conv3D)           (None, 80, 80, 80, 32)    13856     
_________________________________________________________________
leaky_re_lu_64 (LeakyReLU)   (None, 80, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_54 (Batc (None, 80, 80, 80, 32)    128       
__________